In [1]:
import json
import os
import logging
from typing import Dict, Any, List
from datetime import datetime
import random

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from crm_store import CRMStore
from skills.account_opening_tools import *

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Atomic functions testing

In [ ]:
# 0. Delete a client/prospect if already existing in cosmos
try:
    # Example of how to pull from environment variables:
    cosmosdb_endpoint = os.getenv("COSMOSDB_ENDPOINT") or ""
    crm_database_name = os.getenv("COSMOSDB_DATABASE_NAME") or ""
    crm_container_name = os.getenv("COSMOSDB_CONTAINER_CLIENT_NAME") or ""
    key=DefaultAzureCredential()
    
    crm_db = CRMStore(
            url=cosmosdb_endpoint,
            key=key,
            database_name=crm_database_name,
            container_name=crm_container_name
    )

    crm_db.delete_customer_profile("PROSP1400")

except Exception as e:
    logging.error(f"Error in delete_customer_profile: {str(e)}")


ERROR:root:Error in create_prospect: Invalid URL scheme or hostname: SplitResult(scheme='', netloc='', path='', query='', fragment='')


In [4]:

# 1. Create prospect
result = create_prospect("Tom", "Kocinsky", "1976/12/03", "US", "advisor")
print(result)

{"clientID": "PROSP3368", "firstName": "Tom", "lastName": "Kocinsky", "referral_source": "advisor", "id": "PROSP3368", "fullName": "Tom Kocinsky", "dateOfBirth": "1976/12/03", "nationality": "US", "contactDetails": {"email": "", "phone": ""}, "status": "KYC data collected successfully.", "pep_status": false, "risk_level": "", "risk_score": 0, "documents_provided": [], "name_screening_result": "None", "investmentProfile": {"riskProfile": "", "investmentObjectives": "", "investmentHorizon": ""}, "_rid": "TJJPANstunINAAAAAAAAAA==", "_self": "dbs/TJJPAA==/colls/TJJPANstunI=/docs/TJJPANstunINAAAAAAAAAA==/", "_etag": "\"030154fb-0000-4700-0000-67afa5000000\"", "_attachments": "attachments/", "_ts": 1739564288}


In [22]:
prospect_str = fetch_prospect_details("Tom Koc")
print(prospect_str)

{"clientID": "PROSP3368", "firstName": "Tom", "lastName": "Kocinsky", "referral_source": "advisor", "id": "PROSP3368", "fullName": "Tom Kocinsky", "dateOfBirth": "1976/12/03", "nationality": "US", "contactDetails": {"email": "", "phone": ""}, "status": "KYC data collected successfully", "pep_status": false, "risk_level": "", "risk_score": 0, "documents_provided": [], "name_screening_result": "", "investmentProfile": {"riskProfile": "", "investmentObjectives": "", "investmentHorizon": ""}, "_rid": "TJJPANstunINAAAAAAAAAA==", "_self": "dbs/TJJPAA==/colls/TJJPANstunI=/docs/TJJPANstunINAAAAAAAAAA==/", "_etag": "\"000081a2-0000-4700-0000-67d144a50000\"", "_attachments": "attachments/", "declared_source_of_wealth": "", "onboarding": [{"timestamp": "2025-03-11T09:41:01.627152", "step": "First KYC checks passed.", "action": "First KYC checks passed.: Compliance status is First KYC checks passed."}, {"timestamp": "2025-03-11T09:41:08.845308", "step": "Assigned to human review (first line of def

In [ ]:
current_status = "KYC data collected successfully"
upd_prospect = json.loads(prospect_str)
upd_prospect['status'] = current_status
upd_prospect["risk_level"] =  ""
upd_prospect["risk_score"] =  0
upd_prospect["name_screening_result"] = ""
upd_prospect["onboarding"] = []
upd_prospect["declared_source_of_wealth"] =  ""
prospect = update_prospect_details(upd_prospect['clientID'],upd_prospect)

## o1 orchestration 

In [19]:
from openai import AzureOpenAI

# Initialize OpenAI clients
def get_openai_client(key, endpoint, deployment):
    return AzureOpenAI(
        api_key=os.getenv(key),
        api_version="2025-01-01-preview",
        azure_endpoint=os.getenv(endpoint),
        azure_deployment=os.getenv(deployment)
    )

In [20]:
def call_o1(client, scenario):
    current_path = os.getcwd()
    file_path = os.path.join(current_path, 'accountopening/business_logic.txt')
    business_logic = ""
    with open(file_path, 'r') as file:
        business_logic = file.read()

    # Prompt templates
    O1_PROMPT = """
You are a an account opening assistant focusing on orchestrating a full end to end workflow of private banking investement account opening
that involve several steps.
You are a planner. The input you will receive (the scenario) include the current status of the workflow in the field "status" of the prospect_data.
Your task is to review the status and understand which are the next steps to execute next (following the example plan in order below) by creating a plan.

You will have access to an LLM agent that is responsible for executing the plan that you create and will return results.

The LLM agent has access to the following functions:
{tools}

When creating a plan for the LLM to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - **Main actions are marked with numbers** (e.g., 1, 2, 3.).
    - **Sub-actions are under their relevant main actions** 
        - **Sub-actions should start on new lines**
    - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the status was approved, then...').
    - **Include any function parameters from the scenario according to each that are required from each step**
    - **For actions that require using one of the above functions defined**, write a step to call a function using backticks for the function name (e.g., `call the load_from_crm_by_client_fullname function`).
        - Ensure that the proper input arguments are given to the model for instruction. There should not be any ambiguity in the inputs.
    - **The last step** in the instructions should always be calling the `instructions_complete` function. This is necessary so we know the LLM has completed all of the instructions you have given it.
    - **Make the plan simple** Do not add steps on the plan when they are not needed.
    - **Generate summary** Before the `instructions_complete` ask the LLM to make a summary of the actions.
Use markdown format when generating the plan with each step and sub-step.

Please find the scenario below.
{scenario}

---

### Guidance Plan

Below is the ruleset of how you need to structure your final plan, including condition checks, steps/sub-steps, and function calls:
{business_logic}

"""       
        
    prompt= O1_PROMPT.replace("{tools}",str(TOOLS)).replace("{scenario}",str(scenario)).replace("{business_logic}",str(business_logic))
    
    response = client.chat.completions.create(
        model=os.getenv("O1_OPENAI_DEPLOYMENT_NAME"),
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    plan = response.choices[0].message.content
    print(f"📟 Response from o1 plan: {plan}")
    return plan

### Testing o1 planner

In [21]:
o1_client = get_openai_client("O1_OPENAI_API_KEY", "O1_OPENAI_ENDPOINT", "O1_OPENAI_DEPLOYMENT_NAME")

prospect_test = fetch_prospect_details("Tom Koc")

#o1 planner agent part
o1_response = call_o1(o1_client, prospect_test)
o1_reply = {
    'role': 'assistant',
    'name': 'o1 Planner',
    'content': o1_response
}
print( o1_reply )

📟 Response from o1 plan: **Plan**

1. *(Already Completed)* **KYC Information**  
   - The status indicates “KYC data collected successfully,” so this step is complete.

2. **Gather Source of Wealth**  
   - 2.1 `call the collect_sow_info function` with:  
     ```json
     {
       "prospect_data": {
         "clientID": "PROSP3368",
         "firstName": "Tom",
         "lastName": "Kocinsky",
         "dateOfBirth": "1976/12/03",
         "nationality": "US"
       }
     }
     ```
   - 2.2 If the returned status is “SOW information captured,” proceed to step **3**. Else go to step **8**.

3. **Perform Data Management & AI Extraction**  
   - 3.1 `call the perform_data_management_ai_extraction function` with:  
     ```json
     {
       "prospect_data": {
         "clientID": "PROSP3368",
         "documents_provided": []
       }
     }
     ```
   - 3.2 If the returned status is “Documents AI extraction completed,” proceed to step **4**. Else go to step **8**.

4. **Perform Name

Execution (4o-mini) agent

In [9]:
def call_gpt4o(client, plan):
        GPT4O_SYSTEM_PROMPT = """
You are a helpful assistant responsible for executing a plan about account opening.
Your task is to:
1. Follow the plan exactly as written
2. Use the available tools to execute each step
3. Provide clear explanations of what you're doing
4. Always respond with some content explaining your actions
5. Call the instructions_complete function only when all steps are done
6. Never write or execute code
7. In your response, do not add things like "I have succesfully do this and that..." or "This should provide you with the content you asked for..."

PLAN TO EXECUTE:
{plan}

Remember to explain each action you take and provide status updates.
"""
        
        gpt4o_policy_prompt = GPT4O_SYSTEM_PROMPT.replace("{plan}", plan)
        messages = [{'role': 'system', 'content': gpt4o_policy_prompt}]

        while True:
            response = client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                messages=messages,
                tools=TOOLS,
                parallel_tool_calls=False
            )
            #self.logger.info(f" Response from 4o agent:\n {response}")
            
            assistant_message = response.choices[0].message.model_dump()
            messages.append(assistant_message)
   
            if not response.choices[0].message.tool_calls:
                continue

            for tool in response.choices[0].message.tool_calls:
                if tool.function.name == 'instructions_complete':
                    return messages

                function_name = tool.function.name 
              
                print(f"📟 Executing function: {function_name}")
                try:
                    arguments = json.loads(tool.function.arguments)
                    print(f"📟 ...with arguments: {arguments}")
                    function_response = FUNCTION_MAPPING[function_name](**arguments)
                  
                    print( f"{function_name}: {json.dumps(function_response)}")
                    print("Function executed successfully!")
                    print(function_response)
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "content": json.dumps(function_response)
                    })
                    
                except Exception as e:
                    print('error', f"Error in {function_name}: {str(e)}")

## Testing 4o Execution

In [10]:
client = get_openai_client("AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME")

#4o executor part
ex_response = call_gpt4o(client, o1_response)

# Filter assistant messages that have actual content
assistants_4o_contents = [
    msg['content'] for msg in ex_response 
    if msg.get('role') == 'assistant' and msg.get('content') is not None
]

print( f"4o status: {assistants_4o_contents}")

📟 Executing function: collect_sow_info
📟 ...with arguments: {'prospect_data': {'clientID': 'PROSP3368', 'firstName': 'Tom', 'lastName': 'Kocinsky', 'dateOfBirth': '1976/12/03', 'nationality': 'US'}}
collect_sow_info: {"onboarding": [{"timestamp": "2025-03-11T09:40:23.026991", "step": "SOW information captured", "action": "SOW information captured: "}], "status": "SOW information captured"}
Function executed successfully!
{'onboarding': [{'timestamp': '2025-03-11T09:40:23.026991', 'step': 'SOW information captured', 'action': 'SOW information captured: '}], 'status': 'SOW information captured'}
📟 Executing function: perform_data_management_ai_extraction
📟 ...with arguments: {'prospect_data': {'clientID': 'PROSP3368', 'documents_provided': []}}
perform_data_management_ai_extraction: {"extracted_fields": {}, "status": "Documents AI extraction completed"}
Function executed successfully!
{'extracted_fields': {}, 'status': 'Documents AI extraction completed'}
📟 Executing function: perform_na